In [1]:
import requests as rqs
from lxml import etree
from fragrance_net import *
import re
os.chdir("../FragranceNet_Crawler/")

In [2]:
HEADERS = {
    "authority": "www.fragrancenet.com",
    "scheme":"https",
    "accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.90",
    "accept-language": "en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,fr;q=0.6",
    "cookie": 'cookie: MV_SESSION_ID=sAVUmIMKKg9u7jao:rr.com; osid=f15ef2e6e1df7c1157bdfe5aa5b55b5970dbde4e74939c656e843fff7e74bbe1dd8885994b3816fa845886a7d399d963ea3cea2a52ef84e81ece2314d75774ff0919dd99f621909821a1fcdd94ff3ed1982e06f7c2059b4baaceb67f415106ca; currency=currencyUS; affiliate_source=msnus_c_company_terms_fragrancenet_e; __ssid=5d305568ce4e7d027dde1201aa519bc; _fbp=fb.1.1665361570586.205449920; _gcl_au=1.1.156068357.1665361571; _tt_enable_cookie=1; _ttp=c1df9d1c-4ef8-4c3a-b5f5-d333e21817d3; __attentive_id=de4ae5f9f8114a3aa37e8bfd737c3264; __attentive_cco=1665361570947; __attentive_utm_param_source=msn; __attentive_utm_param_medium=cpc; _attn_=eyJ1Ijoie1wiY29cIjoxNjY1MzYxNTcwOTYzLFwidW9cIjoxNjY1MzYxNTcwOTYzLFwibWFcIjoyMTkwMCxcImluXCI6ZmFsc2UsXCJ2YWxcIjpcImRlNGFlNWY5ZjgxMTRhM2FhMzdlOGJmZDczN2MzMjY0XCJ9In0=; __cflb=02DiuCtxEWiftc1CUpjdjPdaPYsm24341yP9Sw25Ny8n8; _gid=GA1.2.942822070.1665626964; _clck=dpbnj9|1|f5o|0; cto_axid=qUesyq71kYYvK24sRohYc4OI3egqoEbR; __attentive_dv=1; __cf_bm=Pl2dwbJyJaAOeJc5WURUFvFfngPtDHH.E97iuRfLOJ8-1665642532-0-AZniagm4upha+ci7w4Zo0yuskgFHp7ipkMW+cUNOXOglfP+3ABCbLLvUODXwU4BWCJq586F2gSbLLSHu7efoDQ7OSRkOInpiTksM+uW+if3/5ImXKp7eHHgAdEuMhMAF87BMw7rYWGb/CWj6LKG8mkQIleTdta5lGC+p9OKddDUV; __attentive_ss_referrer="https://www.fragrancenet.com/cologne?page=5"; amp_365902=OVH-3G9mu-TluZRq4JUBGI...1gf80g11s.1gf818iik.0.7.7; _ga_KQ2KKNE243=GS1.1.1665642532.4.1.1665643336.14.0.0; _ga=GA1.1.1688695203.1665361571; _uetsid=0fb77f204a9c11ed9ee5ebd4ace768a9; _uetvid=24069be0483211edac5f77e39f50f77a; _uetmsclkid=_uet36ecb17ba74c119b88eed238c1d12410; __attentive_pv=2; cto_bundle=IFVTu19UUWE2bHFPMFYlMkJ2Njc3ZDJsd29Tc2J1TUxWRzdQdzZUZk55MDhJJTJGaXNock0xYjVoaXVMUnBVUWhaVWIySEZiNm5KZElKNHk4Ulp0T3ZpYVhKZFZHNndxQlJYYm0lMkJOZzI1ZEhTRFkxOTJLJTJGNGdrbzdUcFkwTWYxJTJGaVJtbUN4WTdQaVNwQWI2ZUc4RyUyRjg5d1k4T2ZudXclM0QlM0Q; _clsk=yw8aec|1665643337421|3|1|d.clarity.ms/collect',
    "device-memory": "8",
    "sec-ch-ua": '"Chromium";v="106", "Microsoft Edge";v="106", "Not;A=Brand";v="99"',
    "sec-ch-platform": '"Windows"',
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.37",
}



In [3]:
base_URL = "https://www.fragrancenet.com/fragrances?page={0}"
max_page = 100
page_pattern = base_URL

In [4]:
option = webdriver.ChromeOptions()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome("./chromedriver.exe", options=option)

In [5]:
all_urls = []
print("Opening Webdriver Page...")
# driver = webdriver.Chrome("./chromedriver.exe")
checker = CheckAntiCrawl()
for page in range(max_page):
    print(f"\r---> Processing Search Page: {page+1}...", end='')
    driver.get(page_pattern.format(page+1))
    checker.check()
    elements = driver.find_elements("xpath", "//div[@id='resultSet']//div[@class='resultItem heightSync']//a[@role='button']")
    for element in elements:
        href = element.get_attribute("href")
        all_urls.append(href)
print(" done.")
print("Webdriver Page closed.")
# return all_urls


Opening Webdriver Page...
---> Processing Search Page: 100... done.
Webdriver Page closed.


In [6]:
items = []
for idx, url in enumerate(all_urls):
    sleep(0.5)
    print(f"\r Processing items: {idx+1}/{len(all_urls)}...", end='')
    driver.get(url)
    checker.check()
    node = etree.HTML(driver.page_source)
    brand = "".join(node.xpath("//*[@id='brandTitle']/text()")).strip()
    infos = node.xpath("//div[@role='option']//*[@class='desktopText']/*[@class='text']/text()")
    prices = node.xpath("//div[@role='option']//*[@class='pricing']/text()")
    ratings = node.xpath("//div[contains(@id, 'star')][@class='reviewRating starRating']/@data-score")

    comment_text = node.xpath("//*[@id='reviewContent']//*[@data-review_id]//*[@class='text']/text()")
    comment_rating=node.xpath("//*[@id='reviewContent']//*[@data-review_id]//*[@data-score]/@data-score")

    comments = list(map(lambda x,y: (x, y), comment_rating, comment_text))

    for idx, info in enumerate(infos):
        sr = re.search("[0-9]+.*[0-9]* oz", info)
        if (sr):
            title = info[:sr.start()].strip()
            size = sr.group()
        else:
            title = info
            size = None
        price = prices[idx]
        items.append({
            "name": title,
            "size": size,
            "brand": brand,
            "scent": None,
            "url": url,
            "price": price,
            "ratings": ratings[0] if len(ratings) else None,
            "comments": comments,
            "platform": 2,
        })
        
print(" done.")

 Processing items: 2400/2400... done.


In [7]:
parser.dump("../../data/fragranceNet.jsonl", items)

5633218